In [ ]:
import pandas as pd
from synapse.ml.predict import MLFlowTransformer
def silver_to_gold(table_name):
    #load
    df = spark.sql("SELECT * FROM silver_lakehouse." + table_name)

    
    #predict
    model = MLFlowTransformer(
        inputCols=['PULocationID', 'DOLocationID'], # Your input columns here
        outputCol="duration_prediction", # Your new column name here
        modelName="nyctaxi-sklearn", # Your model name here
        modelVersion=1 # Your model version here
    )
    df = model.transform(df)
    display(df)
    # #save predictions
    # df.write.mode("overwrite").format('delta').saveAsTable("gold_lakehouse."+table_name)

StatementMeta(, ddf4b2ab-4492-4e09-a8f7-5a0b94a216d6, 3, Finished, Available, Finished)

In [9]:
from mlflow.pyfunc import load_model

# # Replace 'model_path' with the model's URI in MLflow
modelName="nyctaxi-sklearn"
modelVersion=1
model_uri = "models:/{0}/{1}".format(modelName,str(modelVersion))
print(model_uri)
model = load_model(model_uri)


StatementMeta(, 7827766a-40d9-4bec-84cf-1ba20ac1f4b6, 12, Finished, Available, Finished)

models:/nyctaxi-sklearn/1


In [22]:
df = spark.sql("SELECT * FROM silver_lakehouse.green_tripdata_2017")
pdf = df.toPandas()


StatementMeta(, 7827766a-40d9-4bec-84cf-1ba20ac1f4b6, 25, Finished, Available, Finished)

In [23]:
# df.dtypes
pdf['PULocationID'] = pdf['PULocationID'].astype(float)
pdf['DOLocationID'] = pdf['DOLocationID'].astype(float)
final_df = pdf[["PULocationID", "DOLocationID"]]

predictions = model.predict(final_df)
print(predictions)

StatementMeta(, 7827766a-40d9-4bec-84cf-1ba20ac1f4b6, 26, Finished, Available, Finished)

[30.97236141 29.30039328 30.65555857 ... 29.51758351 29.51758351
 29.51758351]


In [ ]:
# now let's process all other tables
for year in range(2017, 2023):
    silver_to_gold("green_tripdata_"+str(year))
    display("green_tripdata_"+str(year))